## knn으로 결측치 처리까지만 한 데이터로 catboost 돌려서 과적합 되는지 확인해보기

In [1]:
import numpy as np
import pandas as pd
import random
import os
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
import random
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint, uniform
from sklearn.preprocessing import LabelEncoder
import optuna
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, log_loss, roc_auc_score, f1_score, mean_squared_error
import numpy as np
import time
from itertools import product
from catboost import CatBoostClassifier, CatBoostRegressor

In [2]:
def seed_everything(seed):
    random.seed(seed) #파이썬 자체 모듈 random 모듈의 시드 고정
    os.environ['PYTHONHASHSEED'] = str(seed) 
    np.random.seed(seed)
    
seed_everything(37) # Seed 고정/

In [3]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [4]:
# knn 알고리즘으로 결측치 처리한 데이터
a_imputed_3 = pd.read_csv('./a_imputed_3.csv')
to_imputed_3 = pd.read_csv('./to_imputed_3.csv')

In [5]:
train_a = train[train.PRODUCT_CODE=='A_31']
train_to = train[(train.PRODUCT_CODE=='T_31')|(train.PRODUCT_CODE=='O_31')]
test_a = test[test.PRODUCT_CODE=='A_31']
test_to = test[(test.PRODUCT_CODE=='T_31')|(test.PRODUCT_CODE=='O_31')]

In [6]:
a_imputed_3.head()

,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,X_128,X_129,X_130,X_131,X_132,...,X_2861,X_2862,X_2863,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870
0,TRAIN_000,1.0,0.533433,2022-06-13 5:14,T050304,7813.0,7813.0,16132.4,16132.4,0.19,...,197.286667,189.0,383.0,368.296296,353.0,39.34,40.89,32.56,34.09,77.77
1,TRAIN_001,2.0,0.541819,2022-06-13 5:22,T050307,11274.0,11274.0,19854.0,19854.0,0.20,...,193.296552,185.6,383.0,367.735849,353.0,38.89,42.82,43.92,35.34,72.55
2,TRAIN_002,1.0,0.531267,2022-06-13 5:30,T050304,7815.0,7815.0,19858.8,19858.8,0.19,...,179.820690,165.5,383.0,367.320755,353.0,39.19,36.65,42.47,36.53,78.35
3,TRAIN_003,2.0,0.537325,2022-06-13 5:39,T050307,11274.0,11274.0,19856.0,19856.0,0.20,...,181.920690,165.8,384.0,369.188679,353.0,37.74,39.17,52.17,30.58,71.78
4,TRAIN_004,1.0,0.531590,2022-06-13 5:47,T050304,7817.0,7817.0,19861.2,19861.2,0.19,...,196.393333,182.6,383.0,367.351852,352.0,38.70,41.89,46.93,33.09,76.97


In [7]:
to_imputed_3.head()

,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_5,X_7,...,X_924,X_925,X_926,X_927,X_928,X_929,X_930,X_931,X_932,X_933
0,TRAIN_022,0.0,0.517719,2022-06-14 8:53,T100304,T_31,2.0,102.0,11.0,45.0,...,172.50,162.80,157.235484,150.90,326.0,309.290909,290.0,13.7,13.443333,13.20
1,TRAIN_023,0.0,0.519090,2022-06-14 9:01,T100304,T_31,2.0,102.0,11.0,45.0,...,176.92,160.18,154.860000,148.34,326.6,309.767273,294.6,13.7,13.453484,13.24
2,TRAIN_025,1.0,0.529362,2022-06-19 9:11,T100304,T_31,2.0,97.0,11.0,45.0,...,177.80,157.10,154.074194,150.90,327.0,310.854546,298.0,13.7,13.454839,13.20
3,TRAIN_026,1.0,0.531992,2022-06-19 9:20,T100306,T_31,2.0,95.0,10.0,54.0,...,166.90,158.50,151.096774,146.50,327.0,313.618182,299.0,13.7,13.461290,13.30
4,TRAIN_029,1.0,0.532405,2022-06-19 23:31,T100304,T_31,2.0,100.0,11.0,45.0,...,177.90,161.30,156.038710,149.10,327.0,312.109091,298.0,13.7,13.422581,13.20


In [11]:
train_a.head()

,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,TRAIN_000,1,0.533433,2022-06-13 5:14,T050304,A_31,NaN,NaN,NaN,NaN,...,39.34,40.89,32.56,34.09,77.77,NaN,NaN,NaN,NaN,NaN
1,TRAIN_001,2,0.541819,2022-06-13 5:22,T050307,A_31,NaN,NaN,NaN,NaN,...,38.89,42.82,43.92,35.34,72.55,NaN,NaN,NaN,NaN,NaN
2,TRAIN_002,1,0.531267,2022-06-13 5:30,T050304,A_31,NaN,NaN,NaN,NaN,...,39.19,36.65,42.47,36.53,78.35,NaN,NaN,NaN,NaN,NaN
3,TRAIN_003,2,0.537325,2022-06-13 5:39,T050307,A_31,NaN,NaN,NaN,NaN,...,37.74,39.17,52.17,30.58,71.78,NaN,NaN,NaN,NaN,NaN
4,TRAIN_004,1,0.531590,2022-06-13 5:47,T050304,A_31,NaN,NaN,NaN,NaN,...,38.70,41.89,46.93,33.09,76.97,NaN,NaN,NaN,NaN,NaN


## train&test 데이터 준비

## A

In [8]:
train_y = train_a.Y_Class

In [9]:
train_x = a_imputed_3.drop(columns = ['PRODUCT_ID', 'Y_Class', 'Y_Quality', 'TIMESTAMP'], axis = 1)
train_x

,LINE,X_128,X_129,X_130,X_131,X_132,X_133,X_134,X_136,X_137,...,X_2861,X_2862,X_2863,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870
0,T050304,7813.0,7813.0,16132.4,16132.4,0.19,0.200,0.19,228.0,228.0,...,197.286667,189.0,383.0,368.296296,353.0,39.340,40.890,32.560,34.090,77.770
1,T050307,11274.0,11274.0,19854.0,19854.0,0.20,0.210,0.20,413.0,414.0,...,193.296552,185.6,383.0,367.735849,353.0,38.890,42.820,43.920,35.340,72.550
2,T050304,7815.0,7815.0,19858.8,19858.8,0.19,0.200,0.19,228.0,228.0,...,179.820690,165.5,383.0,367.320755,353.0,39.190,36.650,42.470,36.530,78.350
3,T050307,11274.0,11274.0,19856.0,19856.0,0.20,0.210,0.20,414.0,414.0,...,181.920690,165.8,384.0,369.188679,353.0,37.740,39.170,52.170,30.580,71.780
4,T050304,7817.0,7817.0,19861.2,19861.2,0.19,0.200,0.18,228.0,228.0,...,196.393333,182.6,383.0,367.351852,352.0,38.700,41.890,46.930,33.090,76.970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,T050304,11920.0,11920.0,1977.8,1977.8,0.19,0.200,0.19,230.0,230.0,...,191.993103,181.6,394.0,371.943396,353.0,51.710,59.640,54.610,57.050,63.180
245,T010305,12586.2,12586.2,1508.4,1508.4,0.19,0.206,0.19,69.0,69.0,...,195.682143,184.9,466.0,448.634615,432.0,53.384,54.966,52.766,56.172,63.228
246,T010306,15655.4,15655.4,1508.4,1508.4,0.19,0.206,0.19,69.0,69.0,...,187.396552,176.7,472.0,450.339623,432.0,53.384,54.966,52.766,56.172,63.228
247,T050304,14810.0,14810.0,3915.2,3915.2,0.19,0.200,0.19,304.0,304.0,...,180.810345,168.7,384.0,369.811321,353.0,49.470,53.070,50.890,55.100,66.490


In [10]:
a_index = test_a.index.tolist()

In [12]:
test_x = test_a
test_x = test_x[train_x.columns]
test_x.head()

,LINE,X_128,X_129,X_130,X_131,X_132,X_133,X_134,X_136,X_137,...,X_2861,X_2862,X_2863,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870
3,T010305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,191.450000,183.8,467.0,444.192308,423.0,NaN,NaN,NaN,NaN,NaN
4,T010306,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,193.082143,179.7,465.0,443.211539,423.0,NaN,NaN,NaN,NaN,NaN
5,T010306,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,192.375862,181.7,466.0,441.830189,422.0,NaN,NaN,NaN,NaN,NaN
6,T010305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,194.351724,175.1,464.0,445.075472,423.0,NaN,NaN,NaN,NaN,NaN
7,T050304,18031.0,18031.0,NaN,NaN,0.19,0.2,0.19,354.0,354.0,...,190.800000,168.3,384.0,369.462963,354.0,57.74,52.51,54.45,57.99,63.16


In [15]:
from catboost import cv, Pool, CatBoostClassifier
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 범주형 특성
categorical_features = ['LINE']

# Pool 생성
train_pool = Pool(data=train_x, label=train_y, cat_features=categorical_features)

# 교차검증 실행
cv_results = cv(
    params={
        'iterations': 1000,
        'learning_rate': 0.03,
        'depth': 3,
        'l2_leaf_reg' : 10,
        'bagging_temperature' : 10,
        'loss_function': 'MultiClass',  # 다중 분류를 위한 손실 함수,
        'verbose': False,
    },
    pool=train_pool,
    fold_count=5,  # 5-겹 교차검증
    shuffle=True,
    partition_random_seed=37,
    stratified=True,
    early_stopping_rounds=50
)

# 교차검증 결과 출력
cv_results_df = pd.DataFrame(cv_results)
print(cv_results_df.head())

# 마지막 반복의 평균 및 표준오차 확인
mean_logloss = cv_results_df['test-MultiClass-mean'].iloc[-1]
std_logloss = cv_results_df['test-MultiClass-std'].iloc[-1]

print(f"최종 평균 Logloss: {mean_logloss:.4f}")
print(f"최종 Logloss 표준오차: {std_logloss:.4f}")

Training on fold [0/5]

bestTest = 0.790285024
bestIteration = 999

Training on fold [1/5]

bestTest = 0.7957136457
bestIteration = 999

Training on fold [2/5]

bestTest = 1.000569099
bestIteration = 190

Training on fold [3/5]

bestTest = 0.8702732234
bestIteration = 845

Training on fold [4/5]

bestTest = 0.8025148318
bestIteration = 978

   iterations  test-MultiClass-mean  test-MultiClass-std  \
0           0              1.095569             0.001396   
1           1              1.093120             0.001719   
2           2              1.090581             0.002669   
3           3              1.086808             0.002590   
4           4              1.084059             0.003001   

   train-MultiClass-mean  train-MultiClass-std  
0               1.094918              0.001944  
1               1.090252              0.002528  
2               1.086744              0.002983  
3               1.083144              0.002615  
4               1.079132              0.003048  
최종

In [16]:
#데이터 분리
# X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, 
#                                                     test_size=0.2, 
#                                                     stratify=train_y, 
#                                                     random_state=37)
# 1. 훈련 데이터와 테스트 데이터 분리
X_train_full, X_test, y_train_full, y_test = train_test_split(
    train_x, train_y, test_size=0.2, stratify=train_y, random_state=37
)

# 2. 훈련 데이터 내에서 검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=0.2, stratify=y_train_full, random_state=37
)

# 최종 모델 학습 (교차검증 이후 전체 데이터로 학습)
final_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.03,
    depth=3,
    l2_leaf_reg = 10,
    bagging_temperature = 10,
    cat_features=categorical_features,
    early_stopping_rounds=50,
    verbose=50
)

final_model.fit(
    X_train,
    y_train,
    eval_set=(X_val, y_val),  # 검증 데이터 설정
    early_stopping_rounds=50,   # 50번 연속 개선되지 않으면 종료
    verbose=50
)

preds_a = final_model.predict(test_x)

0:	learn: 1.0931013	test: 1.0940311	best: 1.0940311 (0)	total: 20.3ms	remaining: 20.2s
50:	learn: 0.9525705	test: 1.0150120	best: 1.0150120 (50)	total: 671ms	remaining: 12.5s
100:	learn: 0.8685705	test: 0.9815274	best: 0.9815274 (100)	total: 1.37s	remaining: 12.2s
150:	learn: 0.8011509	test: 0.9648126	best: 0.9648126 (150)	total: 2.06s	remaining: 11.6s
200:	learn: 0.7492485	test: 0.9509876	best: 0.9509876 (200)	total: 2.74s	remaining: 10.9s
250:	learn: 0.7013962	test: 0.9397767	best: 0.9397671 (248)	total: 3.41s	remaining: 10.2s
300:	learn: 0.6611388	test: 0.9292131	best: 0.9292131 (300)	total: 4.09s	remaining: 9.5s
350:	learn: 0.6271026	test: 0.9223555	best: 0.9218322 (340)	total: 4.75s	remaining: 8.79s
400:	learn: 0.5936542	test: 0.9123609	best: 0.9123609 (400)	total: 5.47s	remaining: 8.18s
450:	learn: 0.5641021	test: 0.9075782	best: 0.9064218 (429)	total: 6.14s	remaining: 7.48s
500:	learn: 0.5385008	test: 0.8988496	best: 0.8987266 (498)	total: 6.81s	remaining: 6.78s
550:	learn: 0.51

In [28]:
y_pred = final_model.predict(X_test)
y_pred_proba = final_model.predict_proba(X_test)

# 테스트 데이터 평가 지표
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average="weighted")
roc_auc = roc_auc_score(y_test, y_pred_proba, multi_class="ovr")
logloss = log_loss(y_test, y_pred_proba)

print(f"Accuracy: {accuracy:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")
print(f"LogLoss: {logloss:.4f}")

Accuracy: 0.5600
F1-Score: 0.4625
ROC-AUC: 0.7163
LogLoss: 0.9476


In [29]:
preds_a = pd.DataFrame(preds_a)
preds_a.index = a_index
preds_a

,0
3,1
4,1
5,1
6,1
7,1
...,...
284,0
285,1
286,1
292,1


In [30]:
preds_a.describe()

,0
count,67.000000
mean,0.805970
std,0.398437
min,0.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,1.000000
